# 🚀 MMM Bayésien - Démonstration des transformations

Ce notebook démontre l'utilisation des transformations d'adstock et de saturation.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.insert(0, '../src')

from models.transformations import (
    geometric_adstock,
    hill_saturation,
    adstock_and_saturation,
    get_adstock_decay_weights,
    get_effective_reach_curve
)

# Configuration plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Démonstration de l'Adstock géométrique

L'adstock modélise la **persistance temporelle** de l'effet publicitaire.

In [ ]:
# Exemple : Campagne TV avec pic de dépenses
weeks = np.arange(20)
spend = np.zeros(20)
spend[5] = 100  # Pic de 100k€ à la semaine 5

# Appliquer différents niveaux de persistance
adstock_low = geometric_adstock(spend, alpha=0.2, l_max=10, normalize=False)
adstock_med = geometric_adstock(spend, alpha=0.5, l_max=10, normalize=False)
adstock_high = geometric_adstock(spend, alpha=0.8, l_max=10, normalize=False)

# Visualisation
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(weeks, spend, 'ko-', label='Dépenses originales', linewidth=2, markersize=8)
ax.plot(weeks, adstock_low, 'b--', label='Adstock α=0.2 (faible persistance)', linewidth=2)
ax.plot(weeks, adstock_med, 'g--', label='Adstock α=0.5 (moyenne)', linewidth=2)
ax.plot(weeks, adstock_high, 'r--', label='Adstock α=0.8 (forte persistance)', linewidth=2)
ax.set_xlabel('Semaine', fontsize=12)
ax.set_ylabel('Effet publicitaire (k€)', fontsize=12)
ax.set_title('Effet de l\'adstock : persistance temporelle de la publicité', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Observation : Plus α est élevé, plus l'effet publicitaire persiste dans le temps.")

## 2. Démonstration de la Saturation de Hill

La saturation modélise les **rendements décroissants** : doubler le budget ne double pas l'impact.

In [ ]:
# Plage de dépenses
spend_range = np.linspace(0, 500, 100)

# Différentes courbes de saturation
sat_fast = hill_saturation(spend_range, half_saturation=50, slope=1.0)
sat_medium = hill_saturation(spend_range, half_saturation=150, slope=1.0)
sat_slow = hill_saturation(spend_range, half_saturation=300, slope=1.0)

# Visualisation
fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(spend_range, sat_fast, 'r-', label='Saturation rapide (k=50)', linewidth=2.5)
ax.plot(spend_range, sat_medium, 'g-', label='Saturation moyenne (k=150)', linewidth=2.5)
ax.plot(spend_range, sat_slow, 'b-', label='Saturation lente (k=300)', linewidth=2.5)
ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Demi-saturation')
ax.set_xlabel('Dépenses publicitaires (k€)', fontsize=12)
ax.set_ylabel('Effet saturé (0-1)', fontsize=12)
ax.set_title('Courbes de saturation : rendements décroissants', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Observation : Plus k est petit, plus la saturation est rapide (marché de niche).")

## 3. Pipeline complet : Adstock + Saturation

Application réaliste sur des données de dépenses hebdomadaires.

In [ ]:
# Données simulées de dépenses TV (20 semaines)
np.random.seed(42)
tv_spend = np.array([120, 150, 180, 200, 90, 70, 100, 130, 160, 140, 
                     110, 95, 85, 120, 150, 170, 190, 80, 60, 100])

# Transformation complète
tv_transformed = adstock_and_saturation(
    tv_spend,
    alpha=0.6,           # Persistance modérée
    half_saturation=120, # Saturation à 120k€
    l_max=6,
    slope=1.0
)

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1 : Évolution temporelle
ax1 = axes[0]
weeks = np.arange(len(tv_spend))
ax1.bar(weeks, tv_spend, alpha=0.5, label='Dépenses brutes', color='steelblue')
ax1.plot(weeks, tv_transformed * 200, 'ro-', label='Effet transformé (×200)', linewidth=2)
ax1.set_xlabel('Semaine', fontsize=12)
ax1.set_ylabel('Valeur', fontsize=12)
ax1.set_title('Transformation complète : dépenses → effet', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2 : Relation dépenses vs effet
ax2 = axes[1]
ax2.scatter(tv_spend, tv_transformed, s=100, alpha=0.6, c=weeks, cmap='viridis')
ax2.set_xlabel('Dépenses TV (k€)', fontsize=12)
ax2.set_ylabel('Effet transformé (0-1)', fontsize=12)
ax2.set_title('Relation dépenses-effet (courbe de réponse)', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
cbar = plt.colorbar(ax2.collections[0], ax=ax2)
cbar.set_label('Semaine', fontsize=10)

plt.tight_layout()
plt.show()

print(f"\n📊 Statistiques :")
print(f"  - Dépenses moyennes : {tv_spend.mean():.1f} k€")
print(f"  - Effet moyen transformé : {tv_transformed.mean():.3f}")
print(f"  - Corrélation dépenses-effet : {np.corrcoef(tv_spend, tv_transformed)[0,1]:.3f}")

## 4. Poids de décroissance de l'adstock

Visualisation des poids de persistance pour différentes valeurs d'alpha.

In [ ]:
# Calculer les poids pour différents alpha
l_max = 10
alphas = [0.2, 0.5, 0.8]

fig, ax = plt.subplots(figsize=(12, 6))

for alpha in alphas:
    weights = get_adstock_decay_weights(alpha, l_max, normalize=True)
    ax.plot(range(l_max + 1), weights, 'o-', label=f'α = {alpha}', linewidth=2, markersize=8)

ax.set_xlabel('Lag (périodes)', fontsize=12)
ax.set_ylabel('Poids normalisé', fontsize=12)
ax.set_title('Poids de décroissance de l\'adstock', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Interprétation :")
print("  - α = 0.2 : Effet de courte durée (marketing digital)")
print("  - α = 0.5 : Effet de durée moyenne (TV, radio)")
print("  - α = 0.8 : Effet de longue durée (branding, sponsoring)")

## 🎯 Prochaines étapes

1. **Charger des données réelles** : Dataset Robyn ou vos propres données
2. **Construire le modèle bayésien** : `src/models/base_mmm.py`
3. **Inférence MCMC** : Échantillonnage avec PyMC
4. **Attribution** : Calculer la contribution de chaque canal
5. **Optimisation** : Recommandations d'allocation budgétaire

---

**📚 Ressources** :
- Documentation complète : `README.md`
- Tests unitaires : `tests/test_transformations.py`
- Configuration : `config/config.yaml`